<a href="https://colab.research.google.com/github/alexiej/laboratory/blob/master/20_01_Keras_API_functional_Embedding_Layers_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

print("Tensorflow version: ", tf.__version__)

!nvidia-smi

TensorFlow 2.x selected.
Tensorflow version:  2.1.0
Fri Feb 21 13:58:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                        

## 01 Define Embedding Layer

* usefull for t high cardinality categorical variable type of data, like day in the week, month in the year, team number, 
* Define input dimension as number of dimensions

More info: https://medium.com/@satnalikamayank12/on-learning-embeddings-for-categorical-data-using-keras-165ff2773fc9


In [6]:
!wget https://s3.amazonaws.com/tripdata/201307-citibike-tripdata.zip
!unzip -o 201307-citibike-tripdata.zip

import pandas as pd
data = pd.read_csv('2013-07 - Citi Bike trip data.csv')
data['weekday'] = pd.to_datetime(data['starttime']).dt.dayofweek

X = data['weekday']
Y = data['tripduration']

data.head()

--2020-02-21 13:59:29--  https://s3.amazonaws.com/tripdata/201307-citibike-tripdata.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.104.93
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.104.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27074629 (26M) [application/zip]
Saving to: ‘201307-citibike-tripdata.zip.2’

201307-citibike-tri 100%[===================>]  25.82M  59.3MB/s    in 0.4s    

2020-02-21 13:59:30 (59.3 MB/s) - ‘201307-citibike-tripdata.zip.2’ saved [27074629/27074629]

Archive:  201307-citibike-tripdata.zip
  inflating: 2013-07 - Citi Bike trip data.csv  


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,weekday
0,634,2013-07-01 00:00:00,2013-07-01 00:10:34,164,E 47 St & 2 Ave,40.753231,-73.970325,504,1 Ave & E 15 St,40.732219,-73.981656,16950,Customer,\N,0,0
1,1547,2013-07-01 00:00:02,2013-07-01 00:25:49,388,W 26 St & 10 Ave,40.749718,-74.002950,459,W 20 St & 11 Ave,40.746745,-74.007756,19816,Customer,\N,0,0
2,178,2013-07-01 00:01:04,2013-07-01 00:04:02,293,Lafayette St & E 8 St,40.730287,-73.990765,237,E 11 St & 2 Ave,40.730473,-73.986724,14548,Subscriber,1980,2,0
3,1580,2013-07-01 00:01:06,2013-07-01 00:27:26,531,Forsyth St & Broome St,40.718939,-73.992663,499,Broadway & W 60 St,40.769155,-73.981918,16063,Customer,\N,0,0
4,757,2013-07-01 00:01:10,2013-07-01 00:13:47,382,University Pl & E 14 St,40.734927,-73.992005,410,Suffolk St & Stanton St,40.720664,-73.985180,19213,Subscriber,1986,1,0


In [0]:
# Imports
from tensorflow.keras.layers import Embedding
from numpy import unique

# Count the unique number of days
n_days = unique(data['weekday']).shape[0]

week_lookup = Embedding(input_dim=n_days,
                        output_dim=1,
                        input_length=1,
                        name='Team-Strength')

In [0]:
# Imports
from tensorflow.keras.layers import Input, Embedding, Flatten
from tensorflow.keras.models import Model

# Create an input layer for the team ID
teamid_in = Input(shape=(1,))

# Lookup the input in the team strength embedding layer
strength_lookup = week_lookup(teamid_in)

# Flatten the output
strength_lookup_flat = Flatten()(strength_lookup)

# Combine the operations into a single, re-usable model
team_strength_model = Model(teamid_in, strength_lookup_flat, name='Team-Strength-Model')

In [0]:
team_strength_model.compile(optimizer='Adam',loss='mse')

In [12]:
team_strength_model.fit(X,Y, validation_split=0.3)

Train on 590391 samples, validate on 253025 samples
590391/590391 [==============================] - 59s 100us/sample - loss: 71964529.3875 - val_loss: 5485380.5578


In [13]:
team_strength_model.summary()

Model: "Team-Strength-Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
Team-Strength (Embedding)    (None, 1, 1)              7         
_________________________________________________________________
flatten (Flatten)            (None, 1)                 0         
Total params: 7
Trainable params: 7
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Import the Subtract layer from keras
from keras.layers import Subtract

# Create a subtract layer using the inputs from the previous exercise
score_diff = Subtract()([team_1_strength, team_2_strength])

In [0]:
# Imports
from keras.layers import Subtract
from keras.models import Model

# Subtraction layer from previous exercise
score_diff = Subtract()([team_1_strength, team_2_strength])

# Create the model
model = Model([team_in_1, team_in_2], score_diff)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

In [14]:
# Get the team_1 column from the regular season data
input_1 = games_season["team_1"]

# Get the team_2 column from the regular season data
input_2 = games_season["team_2"]

# Fit the model to input 1 and 2, using score diff as a target
model.fit([input_1,input_2],
          games_season['score_diff'],
          epochs=1,
          batch_size=2048,
          validation_split=0.1,
          verbose=True)

NameError: ignored

In [0]:
# Get team_1 from the tournament data
input_1 = games_tourney["team_1"]

# Get team_2 from the tournament data
input_2 = games_tourney["team_2"]

# Evaluate the model using these inputs
print(model.evaluate([input_1, input_2], games_tourney["score_diff"], verbose=False))